<a href="https://colab.research.google.com/github/FatimaZahraBoujrad/phishing_classification/blob/main/html_Features_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import json
import gzip
import pandas as pd
from typing import Any, Dict, List, Optional

In [ ]:
ZIP_PATH = "/content/drive/MyDrive/Projet_Phishing/BenignDataset.zip"
DEST_PATH = "/content/BenignDataset"

!unzip -q "$ZIP_PATH" -d "$DEST_PATH"

In [ ]:
ZIP_PATH = "/content/drive/MyDrive/Projet_Phishing/Final_Phishing_Dataset.zip"
DEST_PATH = "/content/PhishingDataset"

!unzip -q "$ZIP_PATH" -d "$DEST_PATH"

In [ ]:
def extract_html(record: Dict[str, Any]) -> Dict[str, Any]:
  rec = record.get("content_info", {}) or {}
  html=rec.get("html", "") or ''
  url = record.get("url", "") or ''
  features={}
  features["url"]=url
  features["html"]=html
  return features

In [ ]:
def read_json_file(file_path: str) -> Optional[Any]:
    """Read JSON from regular or gzipped file"""
    try:
        if file_path.endswith('.gz'):
            with gzip.open(file_path, 'rt', encoding='utf-8') as f:
                return json.load(f)
        else:
            with open(file_path, 'r', encoding='utf-8') as f:
                return json.load(f)
    except Exception as e:
        print(f"⚠️ Error reading {file_path}: {e}")
        return None

def first(lst, default=None):
    """Safely get first element from list"""
    return lst[0] if isinstance(lst, list) and lst else default

In [ ]:


# ============================================================
# FILE PROCESSING
# ============================================================

def get_json_files(folder_path: str, limit: Optional[int] = None) -> List[str]:
    """Get list of JSON files from folder"""
    paths = [
        e.path for e in os.scandir(folder_path)
        if e.is_file() and (e.name.endswith(".json") or e.name.endswith(".json.gz"))
    ]
    paths.sort()

    if limit:
        paths = paths[:limit]

    return paths

def process_file(file_path: str) -> List[Dict[str, Any]]:
    """Process a single JSON file and extract features from all records"""
    rows = []

    obj = read_json_file(file_path)
    if obj is None:
        return rows

    # Handle both single dict and list of dicts
    if isinstance(obj, dict):
        rows.append(extract_html(obj))
    elif isinstance(obj, list):
        for record in obj:
            if isinstance(record, dict):
                rows.append(extract_html(record))

    return rows



In [ ]:
def process_all_files(folder_path: str, limit: Optional[int] = None,
                      progress_every: int = 1000) -> pd.DataFrame:
    """
    Process all JSON files in folder and return DataFrame with extracted features.
    The only hard requirement is that 'url' is the first column; all other features follow automatically.

    Args:
        folder_path: Path to folder containing JSON files
        limit: Maximum number of files to process (None = all)
        progress_every: Print progress every N files

    Returns:
        DataFrame with extracted features
    """
    file_paths = get_json_files(folder_path, limit)

    print(f"📁 Found {len(file_paths)} files to process")
    print(f"🚀 Starting feature extraction...")

    all_rows = []

    for i, file_path in enumerate(file_paths, start=1):
        try:
            rows = process_file(file_path)  # your feature extraction function
            all_rows.extend(rows)

            if i % progress_every == 0:
                print(f"    ✓ Processed {i}/{len(file_paths)} files | {len(all_rows)} records extracted")

        except Exception as e:
            print(f"⚠️ Error processing {file_path}: {e}")

    print(f"✅ Extraction complete: {len(all_rows)} total records from {len(file_paths)} files")

    # Create DataFrame
    df = pd.DataFrame(all_rows)



    return df


In [ ]:
def process_and_save_dataset_to_csv(dataset_folder_path: str, output_csv_file_path: str):
    """
    Processes JSON files in a given folder, extracts URL and HTML content,
    and saves the results into a CSV file.

    Args:
        dataset_folder_path: Path to the folder containing JSON files.
        output_csv_file_path: Path where the output CSV file will be saved.
    """
    print(f"Processing dataset from: {dataset_folder_path}")
    df = process_all_files(dataset_folder_path,limit=20000)

    print(f"Saving processed data to: {output_csv_file_path}")
    df.to_csv(output_csv_file_path, index=False)
    print("Dataset saved successfully.")

print("Defined function `process_and_save_dataset_to_csv`")

Defined function `process_and_save_dataset_to_csv`


In [ ]:
process_and_save_dataset_to_csv('/content/BenignDataset/Benign_Data_BDA', '/content/drive/MyDrive/Projet_Phishing/Dataset/benign_html_data.csv')
#process_and_save_dataset_to_csv('/content/PhishingDataset/Final_Phishing_Dataset', '/content/drive/MyDrive/Projet_Phishing/Dataset/phishing_data_html.csv')

Processing dataset from: /content/BenignDataset/Benign_Data_BDA
📁 Found 20000 files to process
🚀 Starting feature extraction...
    ✓ Processed 1000/20000 files | 1000 records extracted
    ✓ Processed 2000/20000 files | 2000 records extracted
    ✓ Processed 3000/20000 files | 3000 records extracted
    ✓ Processed 4000/20000 files | 4000 records extracted
    ✓ Processed 5000/20000 files | 5000 records extracted
    ✓ Processed 6000/20000 files | 6000 records extracted
    ✓ Processed 7000/20000 files | 7000 records extracted
    ✓ Processed 8000/20000 files | 8000 records extracted
    ✓ Processed 9000/20000 files | 9000 records extracted
    ✓ Processed 10000/20000 files | 10000 records extracted
    ✓ Processed 11000/20000 files | 11000 records extracted
    ✓ Processed 12000/20000 files | 12000 records extracted
    ✓ Processed 13000/20000 files | 13000 records extracted
    ✓ Processed 14000/20000 files | 14000 records extracted
    ✓ Processed 15000/20000 files | 15000 records 

## Extracting  html features


In [ ]:
!pip install beautifulsoup4 lxml tldextract numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
import re
import math
import tldextract
import numpy as np
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import pandas as pd



In [ ]:
def parse_html(html):
    return BeautifulSoup(html, "lxml")


In [ ]:
def extract_form_features(soup, page_url):
    forms = soup.find_all("form")
    features = {
        "num_forms": len(forms),
        "num_password_inputs": 0,
        "external_form_action": 0,
        "empty_form_action": 0,
        "mailto_form_action": 0
    }

    page_domain = tldextract.extract(page_url).domain

    for form in forms:
        action = form.get("action", "").lower()

        # password inputs
        features["num_password_inputs"] += len(
            form.find_all("input", {"type": "password"})
        )

        if action in ["", "#"]:
            features["empty_form_action"] += 1

        if "mailto:" in action:
            features["mailto_form_action"] += 1

        if action.startswith("http"):
            action_domain = tldextract.extract(action).domain
            if action_domain != page_domain:
                features["external_form_action"] += 1

    return features


In [ ]:
def extract_link_features(soup, page_url):
    links = soup.find_all("a", href=True)
    page_domain = tldextract.extract(page_url).domain

    external = empty = ip_links = 0

    for link in links:
        href = link["href"].lower()

        if href in ["#", ""]:
            empty += 1

        if re.match(r"https?://\d+\.\d+\.\d+\.\d+", href):
            ip_links += 1

        if href.startswith("http"):
            domain = tldextract.extract(href).domain
            if domain != page_domain:
                external += 1

    total = len(links) if len(links) else 1

    return {
        "num_links": len(links),
        "external_link_ratio": external / total,
        "empty_link_ratio": empty / total,
        "ip_link_count": ip_links
    }


In [ ]:
def extract_resource_features(soup, page_url):
    tags = soup.find_all(["img", "script", "link"])
    page_domain = tldextract.extract(page_url).domain

    external = 0

    for tag in tags:
        src = tag.get("src") or tag.get("href")
        if src and src.startswith("http"):
            domain = tldextract.extract(src).domain
            if domain != page_domain:
                external += 1

    total = len(tags) if len(tags) else 1

    return {
        "num_resources": len(tags),
        "external_resource_ratio": external / total
    }


In [ ]:
def extract_js_features(soup):
    scripts = soup.find_all("script")
    js_code = " ".join(script.text.lower() for script in scripts)

    suspicious_patterns = {
        "eval": "eval(",
        "document_write": "document.write",
        "settimeout": "settimeout",
        "setinterval": "setinterval",
        "window_location": "window.location",
        "onmouseover": "onmouseover",
    }

    features = {f"uses_{k}": int(v in js_code)
                for k, v in suspicious_patterns.items()}

    features["num_scripts"] = len(scripts)

    return features


In [ ]:
def extract_iframe_features(soup):
    iframes = soup.find_all("iframe")

    hidden = 0
    for iframe in iframes:
        w = iframe.get("width", "")
        h = iframe.get("height", "")
        if w == "0" or h == "0":
            hidden += 1

    return {
        "num_iframes": len(iframes),
        "hidden_iframes": hidden
    }


In [ ]:
def extract_structure_features(html, soup):
    return {
        "html_length": len(html),
        "num_tags": len(soup.find_all())
    }


In [ ]:
def extract_text_features(soup):
    text = soup.get_text().lower()

    phishing_keywords = [
        "login", "verify", "account", "secure",
        "update", "confirm", "password"
    ]

    urgency_words = [
        "urgent", "immediately", "suspended", "restricted"
    ]

    return {
        "phishing_keyword_count": sum(text.count(k) for k in phishing_keywords),
        "urgency_word_count": sum(text.count(u) for u in urgency_words)
    }


In [ ]:
def shannon_entropy(text):
    if not text:
        return 0
    probs = [text.count(c) / len(text) for c in set(text)]
    return -sum(p * math.log2(p) for p in probs)


In [ ]:
def extract_entropy_features(soup):
    scripts = soup.find_all("script")
    entropies = []

    obfuscation_indicators = 0

    for script in scripts:
        js = script.text.strip()
        if not js:
            continue

        entropy = shannon_entropy(js)
        entropies.append(entropy)

        # heuristics for obfuscation
        if entropy > 4.5:
            obfuscation_indicators += 1
        if re.search(r"(\\x[0-9a-f]{2})", js):
            obfuscation_indicators += 1
        if re.search(r"(atob\(|btoa\()", js):
            obfuscation_indicators += 1

    return {
        "avg_js_entropy": np.mean(entropies) if entropies else 0,
        "max_js_entropy": max(entropies) if entropies else 0,
        "obfuscated_js_count": obfuscation_indicators
    }


In [ ]:
def extract_input_features(soup):
    inputs = soup.find_all("input")
    hidden = soup.find_all("input", {"type": "hidden"})

    return {
        "input_count": len(inputs),
        "hidden_fields_count": len(hidden),
        "has_sensitive_fields": int(
            any(i.get("type") in ["password", "email"] for i in inputs)
        )
    }


In [ ]:
def extract_marquee_feature(soup):
    return {
        "has_marquee": int(bool(soup.find("marquee")))
    }


In [ ]:
def extract_hidden_redirect(soup):
    meta = soup.find("meta", attrs={"http-equiv": "refresh"})
    return {
        "has_hidden_redirect": int(meta is not None)
    }


In [ ]:
def extract_image_features(soup, page_url):
    images = soup.find_all("img")
    page_domain = tldextract.extract(page_url).domain

    external = cdn = 0
    cdn_keywords = ["cdn", "cloudflare", "akamai", "aws"]

    for img in images:
        src = img.get("src", "")
        if src.startswith("http"):
            domain = tldextract.extract(src).domain
            if domain != page_domain:
                external += 1
            if any(k in src for k in cdn_keywords):
                cdn += 1

    return {
        "image_count": len(images),
        "external_images_count": external,
        "cdn_images_count": cdn
    }


In [ ]:
def extract_brand_impersonation(soup):
    text = soup.get_text().lower()
    brands = ["paypal", "google", "facebook", "microsoft", "amazon"]

    return {
        "brand_impersonation": int(any(b in text for b in brands))
    }


In [ ]:
def extract_facebook_cookie_harvesting(soup):
    scripts = soup.find_all("script")
    js = " ".join(s.text.lower() for s in scripts)

    return {
        "facebook_cookie_harvesting": int(
            "facebook.com/tr" in js or "document.cookie" in js
        )
    }


In [ ]:
def extract_all_features(html, url):
    # Ensure html is a string, handle potential NaN values
    html_content = str(html) if pd.isna(html) else html
    soup = parse_html(html_content)

    features = {}
    features.update(extract_form_features(soup, url))
    features.update(extract_link_features(soup, url))
    features.update(extract_resource_features(soup, url))
    features.update(extract_js_features(soup))
    features.update(extract_iframe_features(soup))
    features.update(extract_structure_features(html_content, soup))
    features.update(extract_text_features(soup))
    features.update(extract_entropy_features(soup))
    features.update(extract_input_features(soup))
    features.update(extract_marquee_feature(soup))
    features.update(extract_hidden_redirect(soup))
    features.update(extract_image_features(soup, url))
    features.update(extract_brand_impersonation(soup))
    features.update(extract_facebook_cookie_harvesting(soup))

    return features

In [ ]:
phishing = pd.read_csv(
    '/content/drive/MyDrive/Projet_Phishing/Dataset/phishing_data_html.csv',
    skiprows=range(1, 10001),  # saute les 10 000 premières lignes (en gardant l'en-tête)
    nrows=10000               # lit les 10 000 suivantes
)



In [ ]:
phishing_features = phishing.apply(lambda row: extract_all_features(row['html'], row['url']), axis=1)
phishing_html = pd.DataFrame(phishing_features.tolist())

In [ ]:
phishing_html.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   num_forms                   10000 non-null  int64  
 1   num_password_inputs         10000 non-null  int64  
 2   external_form_action        10000 non-null  int64  
 3   empty_form_action           10000 non-null  int64  
 4   mailto_form_action          10000 non-null  int64  
 5   num_links                   10000 non-null  int64  
 6   external_link_ratio         10000 non-null  float64
 7   empty_link_ratio            10000 non-null  float64
 8   ip_link_count               10000 non-null  int64  
 9   num_resources               10000 non-null  int64  
 10  external_resource_ratio     10000 non-null  float64
 11  uses_eval                   10000 non-null  int64  
 12  uses_document_write         10000 non-null  int64  
 13  uses_settimeout             1000

In [ ]:
phishing_html.head()

,num_forms,num_password_inputs,external_form_action,empty_form_action,mailto_form_action,num_links,external_link_ratio,empty_link_ratio,ip_link_count,num_resources,...,input_count,hidden_fields_count,has_sensitive_fields,has_marquee,has_hidden_redirect,image_count,external_images_count,cdn_images_count,brand_impersonation,facebook_cookie_harvesting
0,1,0,0,0,0,1,0.000000,0.000000,0,57,...,7,5,0,0,0,5,0,0,0,0
1,1,1,1,0,0,2,0.500000,0.500000,0,17,...,4,2,1,0,0,1,1,0,1,0
2,8,0,8,0,0,220,0.881818,0.031818,0,138,...,15,0,0,0,0,54,45,0,0,1
3,0,0,0,0,0,1,1.000000,0.000000,0,9,...,0,0,0,0,0,1,0,0,1,0
4,1,0,0,1,0,127,0.984252,0.000000,0,49,...,0,0,0,0,0,28,0,0,0,0


In [ ]:
phishing_html.describe()

,num_forms,num_password_inputs,external_form_action,empty_form_action,mailto_form_action,num_links,external_link_ratio,empty_link_ratio,ip_link_count,num_resources,...,input_count,hidden_fields_count,has_sensitive_fields,has_marquee,has_hidden_redirect,image_count,external_images_count,cdn_images_count,brand_impersonation,facebook_cookie_harvesting
count,10000.000000,10000.00000,10000.00000,10000.000000,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.446500,0.12890,0.66200,0.273600,0.0,62.470400,0.304625,0.092042,0.009700,42.345000,...,8.601800,2.617800,0.195300,0.005000,0.069600,25.664900,6.980300,3.942800,0.224400,0.077000
std,2.329867,1.33344,2.26311,0.779104,0.0,169.350148,0.407268,0.250441,0.240024,78.503103,...,48.552157,6.220466,0.396451,0.070537,0.254484,73.809742,14.622951,10.790105,0.417207,0.266605
min,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.00000,0.00000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,9.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.00000,0.00000,0.000000,0.0,15.000000,0.000000,0.000000,0.000000,32.000000,...,4.000000,0.000000,0.000000,0.000000,0.000000,8.000000,1.000000,0.000000,0.000000,0.000000
75%,1.000000,0.00000,0.00000,0.000000,0.0,71.000000,0.826087,0.004630,0.000000,56.000000,...,6.000000,3.000000,0.000000,0.000000,0.000000,37.000000,7.000000,1.000000,0.000000,0.000000
max,32.000000,90.00000,10.00000,32.000000,0.0,3363.000000,1.000000,1.000000,6.000000,1666.000000,...,1125.000000,168.000000,1.000000,1.000000,1.000000,1610.000000,404.000000,404.000000,1.000000,1.000000


In [ ]:
phishing_html.to_csv('/content/drive/MyDrive/Projet_Phishing/Dataset/phishing_html_features.csv',  mode='a',
    header=False,
    index=False)

In [ ]:
phishing_html = pd.read_csv('/content/drive/MyDrive/Projet_Phishing/Dataset/phishing_html_features.csv')
phishing_html.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   num_forms                   20000 non-null  int64  
 1   num_password_inputs         20000 non-null  int64  
 2   external_form_action        20000 non-null  int64  
 3   empty_form_action           20000 non-null  int64  
 4   mailto_form_action          20000 non-null  int64  
 5   num_links                   20000 non-null  int64  
 6   external_link_ratio         20000 non-null  float64
 7   empty_link_ratio            20000 non-null  float64
 8   ip_link_count               20000 non-null  int64  
 9   num_resources               20000 non-null  int64  
 10  external_resource_ratio     20000 non-null  float64
 11  uses_eval                   20000 non-null  int64  
 12  uses_document_write         20000 non-null  int64  
 13  uses_settimeout             200

In [ ]:
bening=pd.read_csv('/content/drive/MyDrive/Projet_Phishing/Dataset/benign_html_data.csv',skiprows=range(1, 10001),  # saute les 10 000 premières lignes (en gardant l'en-tête)
    nrows=10000  )             # lit les 10 000 suivantes
           # lit les 10 000 suivantes


In [ ]:
bening_features = bening.apply(lambda row: extract_all_features(row['html'], row['url']), axis=1)
bening_html = pd.DataFrame(bening_features.tolist())

In [ ]:
bening_html.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   num_forms                   10000 non-null  int64  
 1   num_password_inputs         10000 non-null  int64  
 2   external_form_action        10000 non-null  int64  
 3   empty_form_action           10000 non-null  int64  
 4   mailto_form_action          10000 non-null  int64  
 5   num_links                   10000 non-null  int64  
 6   external_link_ratio         10000 non-null  float64
 7   empty_link_ratio            10000 non-null  float64
 8   ip_link_count               10000 non-null  int64  
 9   num_resources               10000 non-null  int64  
 10  external_resource_ratio     10000 non-null  float64
 11  uses_eval                   10000 non-null  int64  
 12  uses_document_write         10000 non-null  int64  
 13  uses_settimeout             1000

In [ ]:
bening_html.describe()

,num_forms,num_password_inputs,external_form_action,empty_form_action,mailto_form_action,num_links,external_link_ratio,empty_link_ratio,ip_link_count,num_resources,...,input_count,hidden_fields_count,has_sensitive_fields,has_marquee,has_hidden_redirect,image_count,external_images_count,cdn_images_count,brand_impersonation,facebook_cookie_harvesting
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.289400,0.205700,0.052000,0.493700,0.0,130.640100,0.113267,0.055197,0.003200,59.556700,...,21.468900,2.736300,0.257300,0.00210,0.043600,22.832500,5.317900,3.574000,0.360900,0.202100
std,2.737517,0.598852,0.297834,1.773661,0.0,306.054646,0.197906,0.167976,0.169094,102.668069,...,100.584633,11.121741,0.437168,0.04578,0.204214,86.710553,37.680804,38.381228,0.480286,0.401587
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,3.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.0,38.000000,0.031250,0.000000,0.000000,38.000000,...,2.000000,0.000000,0.000000,0.00000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,0.000000,0.000000,1.000000,0.0,140.000000,0.132096,0.018757,0.000000,81.000000,...,8.000000,2.000000,1.000000,0.00000,0.000000,21.000000,2.000000,0.000000,1.000000,0.000000
max,91.000000,17.000000,6.000000,73.000000,0.0,4941.000000,1.000000,1.000000,14.000000,2316.000000,...,1731.000000,296.000000,1.000000,1.00000,1.000000,2229.000000,2226.000000,2221.000000,1.000000,1.000000


In [ ]:
bening_html.to_csv('/content/drive/MyDrive/Projet_Phishing/Dataset/bening_html_features.csv',  mode='a',
    header=False,
    index=False)

In [ ]:
bening_html = pd.read_csv('/content/drive/MyDrive/Projet_Phishing/Dataset/bening_html_features.csv')
bening_html.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   num_forms                   20000 non-null  int64  
 1   num_password_inputs         20000 non-null  int64  
 2   external_form_action        20000 non-null  int64  
 3   empty_form_action           20000 non-null  int64  
 4   mailto_form_action          20000 non-null  int64  
 5   num_links                   20000 non-null  int64  
 6   external_link_ratio         20000 non-null  float64
 7   empty_link_ratio            20000 non-null  float64
 8   ip_link_count               20000 non-null  int64  
 9   num_resources               20000 non-null  int64  
 10  external_resource_ratio     20000 non-null  float64
 11  uses_eval                   20000 non-null  int64  
 12  uses_document_write         20000 non-null  int64  
 13  uses_settimeout             200